신경망
========
층 하나짜리 간단한 신경망과, 3층짜리 신경망을 구현해보자. 그리고 출력층에 쓸 소프트맥스 함수를 만들어보자

### 간단한 신경망 구현

레이어 하나짜리 신경망은 `Y = AX` 로 표현할 수 있음

![](img/3-simple-network.jpg)

In [1]:
import numpy as np

def simple_network(x):
    return x @ np.array([
        [1, 3, 5],
        [2, 4, 6],
    ])

do = lambda *args: (lambda x: print(f'x: {x}\ty: {simple_network(x)}'))(np.array(args))
do(1, 0)
do(0, 1)
do(2, 3)

x: [1 0]	y: [1 3 5]
x: [0 1]	y: [2 4 6]
x: [2 3]	y: [ 8 18 28]


### 죠금 더 복잡한 신경망 구현

위에꺼를 여러번 겹치면 레이어 여러개가 된다. 근데 편향을 넣으면 `Y = AX`가 `Y = AX + B`가 됨. 레이어마다 액티베이션 펑션을 넣으면 `Z = f(AX + B)`가 됨.

![](img/3-network.jpg)

In [2]:
from common import sigmoid

ident = lambda x: x

def network(X):
    # Network constants
    W1 = np.array([
        [0.1, 0.3, 0.5],
        [0.2, 0.4, 0.6],
    ])
    B1 = np.array([0.1, 0.2, 0.3])

    W2 = np.array([
        [0.1, 0.4],
        [0.2, 0.5],
        [0.3, 0.6],
    ])
    B2 = np.array([0.1, 0.2])

    W3 = np.array([
        [0.1, 0.3],
        [0.2, 0.4],
    ])
    B3 = np.array([0.1, 0.2])

    # A = XW + B
    Z1 = sigmoid(X@W1 + B1)
    Z2 = sigmoid(Z1@W2 + B2)
    Y = ident(Z2@W3 + B3)
    
    return Y, [X, Z1, Z2, Y]

def do(fn):
    result, layers = fn(np.array([1.0, 0.5]))
    print(f'결과 : {result}')
    print('중간레이어 :')
    print('\n'.join([f'  - {str(layer)}' for layer in layers]))
    print()
    
do(network)

결과 : [ 0.31682708  0.69627909]
중간레이어 :
  - [ 1.   0.5]
  - [ 0.57444252  0.66818777  0.75026011]
  - [ 0.62624937  0.7710107 ]
  - [ 0.31682708  0.69627909]



이때 비동차 벡터를 쓰면 레이어 하나를 어파인 변환(`Y = AX + B`)이 아니라 선형 변환(`Y' = A'X'`)으로 만들 수 있지 않을까?

In [3]:
def network_homogeneous(X_orig):
    # Change input as a homogeneous vector
    X = np.append(X_orig, 1)
    
    # Network constants
    W1 = np.array([
        [0.1, 0.3, 0.5, 0],
        [0.2, 0.4, 0.6, 0],
        [0.1, 0.2, 0.3, 1],
    ])
    W2 = np.array([
        [0.1, 0.4, 0],
        [0.2, 0.5, 0],
        [0.3, 0.6, 0],
        [0.1, 0.2, 1],
    ])
    W3 = np.array([
        [0.1, 0.3, 0],
        [0.2, 0.4, 0],
        [0.1, 0.2, 1],
    ])
    
    # A' = X'W'
    Z1 = sigmoid(X @ W1)
    Z1[-1] = 1
    Z2 = sigmoid(Z1 @ W2)
    Z2[-1] = 1
    Y = ident(Z2 @ W3)    
    
    return Y[0:-1], [X, Z1, Z2, Y]


do(network)
do(network_homogeneous)

결과 : [ 0.31682708  0.69627909]
중간레이어 :
  - [ 1.   0.5]
  - [ 0.57444252  0.66818777  0.75026011]
  - [ 0.62624937  0.7710107 ]
  - [ 0.31682708  0.69627909]

결과 : [ 0.31682708  0.69627909]
중간레이어 :
  - [ 1.   0.5  1. ]
  - [ 0.57444252  0.66818777  0.75026011  1.        ]
  - [ 0.62624937  0.7710107   1.        ]
  - [ 0.31682708  0.69627909  1.        ]



### 소프트맥스 구현

출력층에서는 은닉층과 다른 함수를 쓴다. 리그레션에서는 항등함수를, 분류에서는 소프트맥스를 쓴다고함. 소프트맥스를 짜보자.

<!-- https://www.codecogs.com/latex/eqneditor.php -->

$$y_k = \frac{exp(a_k)}{\sum_{i=1}^{n}exp(a_i)}$$

In [4]:
def softmax_naive(A):
    ExpA = np.exp(A)
    return ExpA / ExpA.sum()
    
softmax_naive(np.array([0.3, 2.9, 4.0]))

array([ 0.01821127,  0.24519181,  0.73659691])

이때 분자 부모에 같은 값의 상수를 더해서, 부동소수점 오차를 줄일 수 있다.

$$
\begin{align*}
y_k = \frac{exp(a_k)}{\sum_{i=1}^{n}exp(a_i)} &= \frac{Cexp(a_k)}{C\sum_{i=1}^{n}exp(a_i)} \\
&= \frac{exp(a_k + logC)}{\sum_{i=1}^{n}exp(a_i + logC)} \\
&= \frac{exp(a_k + C')}{\sum_{i=1}^{n}exp(a_i + C')}
\end{align*}
$$

In [5]:
def softmax_1d(A):
    ExpA = np.exp(A - A.max())
    return ExpA / ExpA.sum()

# 두 함수의 결과가 같다
print(softmax_naive(np.array([0.3, 2.9, 4.0])))
print(softmax_1d(np.array([0.3, 2.9, 4.0])))

[ 0.01821127  0.24519181  0.73659691]
[ 0.01821127  0.24519181  0.73659691]


약간의 처리를 더 해주면, 입력이 하나짜리 1차원 배열이 아니라 다차원 배열로 들어와도 (batch로 입력되어도) 일관적으로 계산해줄 수 있다.

In [6]:
def softmax(A):
    extend = (lambda x:x) if A.ndim == 1 else (lambda x: x[..., np.newaxis])
    ExpA = np.exp(A - extend(A.max(axis=-1)))
    return ExpA / extend(ExpA.sum(axis=-1))

do = lambda x: print(softmax(np.array(x)))

do([1, 2])
do([4, 8])
do([4, 0])
do([7, 2])
print()
do([
    [1, 2],
    [4, 8],
])
do([
    [4, 0],
    [7, 2],
])
print()
do([[
    [1, 2],
    [4, 8],
], [
    [4, 0],
    [7, 2],
]])

[ 0.26894142  0.73105858]
[ 0.01798621  0.98201379]
[ 0.98201379  0.01798621]
[ 0.99330715  0.00669285]

[[ 0.26894142  0.73105858]
 [ 0.01798621  0.98201379]]
[[ 0.98201379  0.01798621]
 [ 0.99330715  0.00669285]]

[[[ 0.26894142  0.73105858]
  [ 0.01798621  0.98201379]]

 [[ 0.98201379  0.01798621]
  [ 0.99330715  0.00669285]]]
